In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
df=pd.read_csv('.\processed_EPS1.csv',index_col=0)
df

In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor,TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np



import os
import wget
from pathlib import Path

In [ ]:
df1 = df.drop(['D2', 'D0', 'Validity', 'ipk', 'irms', 'Vo', 'nZVS'], axis=1)
df1 = df1.reset_index(drop=True)

In [ ]:
df1

In [ ]:
target = 'Ptotal'

df1["Set"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df1.shape[0],))

train_indices = df1[df1.Set=="train"].index
valid_indices = df1[df1.Set=="valid"].index
test_indices = df1[df1.Set=="test"].index

In [ ]:
tabnet_params = { "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=0.1),
                 "scheduler_params": {"mode": 'max', "factor": 0.5, "patience": 30, "verbose": True},
                 "scheduler_fn":torch.optim.lr_scheduler.ReduceLROnPlateau,
                 "n_d":32,
                 "n_a":32,
                 #"n_steps":5,
                 "n_independent":5,
                 "n_shared":5
                 
                # "mask_type":'entmax', # "sparsemax"
                # "grouped_features" : grouped_features
                }


clf = TabNetRegressor(n_d=12,n_a=12 )

In [ ]:
unused_feat = ['Set']
features = [ col for col in df1.columns if col not in unused_feat+[target]] 

In [ ]:
X_train = df1[features].values[train_indices]
y_train = df1[target].values[train_indices].reshape(-1, 1)

X_valid = df1[features].values[valid_indices]
y_valid = df1[target].values[valid_indices].reshape(-1, 1)

X_test = df1[features].values[test_indices]
y_test = df1[target].values[test_indices].reshape(-1, 1)

In [ ]:
max_epochs = 600 if not os.getenv("CI", False) else 2

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    eval_name=['train', 'valid'],
    eval_metric=['mse'],
    max_epochs=max_epochs,
    patience=300,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    #augmentations=aug
) 

In [ ]:
preds = clf.predict(X_test)

y_true = y_test

test_score = mean_squared_error(y_pred=preds, y_true=y_true)
test_score

In [ ]:
clf.feature_importances_

In [ ]:
clf.feature_importance=clf.feature_importances_.reshape(1,3)

In [ ]:
clf.feature_importance

In [ ]:
explain_matrix, masks = clf.explain(X_test)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, axs = plt.subplots(1, 1, figsize=(20, 20))
axs.imshow(clf.feature_importance)
axs.set_title(f"feature importance")


In [ ]:
masks[1].shape

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, axs = plt.subplots(1, 3, figsize=(20,20))

for i in range(3):
    axs[i].imshow(masks[i][1000:1001])
    axs[i].set_title(f"mask {i}")
    axs[i].set_xticklabels(labels = features, rotation=45)

In [ ]:
import shap
# 假设你知道列名
column_names = ['P', 'Vref', 'D1']
temp_X_test = pd.DataFrame(X_test, columns=column_names)
print(temp_X_test.columns)

explainer = shap.Explainer(clf.predict,X_test)
shap_values=explainer(X_test)

In [ ]:
shap_values.feature_names=column_names

In [ ]:
shap.plots.waterfall(shap_values[1000])

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0],matplotlib=True)

In [ ]:
shap.summary_plot(shap_values,X_test)

In [ ]:
shap.plots.scatter(shap_values[:, 0])

In [ ]:
shap.dependence_plot('P', shap_values.values,temp_X_test, interaction_index=None)

In [ ]:
shap.dependence_plot('Vref', shap_values.values,temp_X_test, interaction_index='P')


In [ ]:
shap.initjs()
shap.plots.force(shap_values.base_values[0],shap_values.values)

In [ ]:
shap_values.base_values[0]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
# 定义随机森林模型
model = RandomForestRegressor()

# 定义超参数网格
# 定义超参数网格
param_grid = {
    'n_estimators': [30, 50, 100],
    'max_depth': [5, 8, 10],
    'min_samples_split': [2, 5, 10],
}


# 使用网格搜索进行超参数调优
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 获取最佳模型
best_model = grid_search.best_estimator_

# 在测试集上进行预测
Ptotal_test_pred = best_model.predict(X_test)

# 计算测试集上的MSE和MAE
mse_test = mean_squared_error(y_test, Ptotal_test_pred)
mape_test = mean_absolute_percentage_error(y_test, Ptotal_test_pred)

print(f"Best Model - MSE on Test Set: {mse_test:.4f} - MAPE on Test Set: {mape_test:.4f}")

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# 定义XGBoost回归模型
model = xgb.XGBRegressor()

# 定义超参数网格
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
}

# 使用网格搜索进行超参数调优
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 获取最佳模型
best_model1 = grid_search.best_estimator_

# 在测试集上进行预测
Ptotal_test_pred = best_model1.predict(X_test)

# 计算测试集上的MSE和MAE
mse_test = mean_squared_error(y_test, Ptotal_test_pred)
mape_test = mean_absolute_percentage_error(y_test, Ptotal_test_pred)

print(f"Best Model - MSE on Test Set: {mse_test:.4f} - MAPE on Test Set: {mape_test:.4f}")

In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(X_train)
inputs_train=scaler_inputs.transform(X_train)
inputs_train.shape

In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(X_valid)
inputs_valid=scaler_inputs.transform(X_valid)
inputs_valid.shape

In [ ]:
scaler_inputs=MinMaxScaler()
scaler_inputs.fit(X_test)
inputs_test=scaler_inputs.transform(X_test)
inputs_test.shape

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

#  定义参数网格

# 创建支持向量回归模型
model = SVR()

# 定义超参数网格
param_grid = {
    'C': [1,10, 100],
    'kernel': ['linear',  'rbf'],
}

# 使用网格搜索进行超参数调优
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(inputs_train, y_train)

# 获取最佳模型
best_model1 = grid_search.best_estimator_
best_params = grid_search.best_params_

# 输出最佳参数
print(f"Best Parameters: {best_params}")


# 在测试集上进行预测
Ptotal_test_pred = best_model1.predict(inputs_test)

# 计算测试集上的MSE和MAE
mse_test = mean_squared_error(y_test, Ptotal_test_pred)
mape_test = mean_absolute_percentage_error(y_test, Ptotal_test_pred)

print(f"Best Model - MSE on Test Set: {mse_test:.4f} - MAPE on Test Set: {mape_test:.4f}")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math,copy,time
from torch.autograd import Variable
class Single_Residual(nn.Module):
    def __init__(self,input_size,hidden_size,dropout,final=False):
        super(Single_Residual,self).__init__()
        self.linear_layer=nn.Linear(input_size,hidden_size)
        self.final=final
        if not self.final:
            self.batch_norm=nn.BatchNorm1d(hidden_size)
            self.dropout=nn.Dropout(dropout)
        self.residual_maker=nn.Parameter(torch.zeros(input_size,hidden_size))
        self.residual_maker.requires_grad_(False)
        if hidden_size<=input_size:
            self.residual_maker[torch.randperm(input_size)[:hidden_size],torch.arange(0,hidden_size)]=1
        else:
            self.residual_maker[torch.arange(0,input_size),torch.randperm(hidden_size)[:input_size]] = 1
            
    def forward(self,x):
        if self.final:
            h1=self.linear_layer(x)
        else:
            h1=F.relu(self.linear_layer(x))
        return h1

class Residual_DNN(nn.Module):
    def __init__(self,input_size,hidden_sizes,dropout):
        super(Residual_DNN,self).__init__()
        assert isinstance(hidden_sizes,list),'the "hidden_sizes" should be list '
        hidden_num=len(hidden_sizes)
        self.residuals=[f"self.residual{i}"for i in range(hidden_num)]
        for i in range(hidden_num):
            if i==hidden_num-1:
                exec(self.residuals[i]+"=Single_Residual(input_size,hidden_sizes[i],dropout,True)")
            else:
                exec(self.residuals[i]+"=Single_Residual(input_size,hidden_sizes[i],dropout)")
            input_size=hidden_sizes[i]
    def forward(self,x):
        for residual in self.residuals:
            x=eval(residual)(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

# 数据类型转换
inputs_train = torch.tensor(inputs_train).float()
y_train = torch.tensor(y_train).float()
inputs_valid = torch.tensor(inputs_valid).float()
y_valid = torch.tensor(y_valid).float()
inputs_test = torch.tensor(inputs_test).float()
y_test = torch.tensor(y_test).float()

# 创建 DataLoader
train_data = TensorDataset(inputs_train, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=12, shuffle=True)

# 定义模型
input_size = inputs_train.shape[1]
hidden_sizes = [256, 128,64,1]  # 调整隐藏层大小
dropout = 0  # 调整丢弃率
model = Residual_DNN(input_size, hidden_sizes, dropout)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.08)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=30, factor=0.5, verbose=True)

# 训练模型
epochs =500
best_mse = float('inf')
early_stopping_patience = 150
best_model_state = None
patience = 0

for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # 评估模型性能
    model.eval()
    with torch.no_grad():
        valid_pred = model(inputs_valid)
        mse = mean_squared_error(y_valid, valid_pred)
        print(f"Epoch [{epoch+1}/{epochs}] - Validation MSE: {mse:.4f}")
        
        # 学习率调度
        scheduler.step(mse)
        
        # 检查并保存最佳模型
        if mse < best_mse:
            best_mse = mse
            best_model_state = model.state_dict().copy()
            patience = 0
        else:
            patience += 1
            if patience > early_stopping_patience:
                print("Early stopping")
                break

# 加载最佳模型
model.load_state_dict(best_model_state)

# 在测试集上计算MSE
model.eval()
with torch.no_grad():
    test_pred = model(inputs_test)
    test_mse = mean_squared_error(y_test, test_pred)
    test_mape = mean_absolute_percentage_error(y_test, test_pred)
    
    print(f"Test MSE with best model: {test_mse:.4f}")
    print(f"Test MAPE with best model: {test_mape:.4f}")